In [1]:
import math
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

In [2]:
def idf(word, count_list):
    return math.log(len(count_list)) / (1 + dj.get(word))

In [3]:
def tf(word, count):
    return count[word] / sum(count.values())

In [15]:
def u(word):
    try:
        return (math.sqrt(math.pow((dj.get(word)/2-dn.get(word)),2)+math.pow((dj.get(word)/2-da.get(word)),2)))/2
    except Exception:
        return 1

In [5]:
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)*u(word)

In [6]:
def get_countlist(path):
    file = open(path, 'r',encoding='utf-8')
    lines = file.read()
    # 直接用splitlines（）放法来实现行分割
    other_result = lines.splitlines()
    countlist = []
    for text in other_result:
        text = str(text)
        countlist.append(count_term(text))
    return countlist
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed
def get_tokens(text):
    #  lower() 方法转换字符串中所有大写字符为小写。
    # print(text)
    lower = text.lower()
    # print("lower==",lower)
    # 删除标点符号
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lower.translate(remove_punctuation_map)
    # print("no_punctuation==",no_punctuation)
    tokens = nltk.word_tokenize(no_punctuation)#分词
    # print(tokens)
    return tokens
#计算词频
def count_term(text):
    tokens = get_tokens(text)
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    stemmer = PorterStemmer()
    stemmed = stem_tokens(filtered, stemmer)
    # 计算词频
    count = Counter(stemmed)
    return count

In [7]:
def getVector(text,model,count_list):
    sentence = []
    doc_vec = []
    text = text.split()
#     word_list = []
#     word_list.append(text.split(' '))
#     print(word_list)
    dic = {}
    for word in text:
        if word not in dic:
            dic[word] = 1
        else:
            dic[word] = dic[word] + 1
        word_vec = tfidf(word,dic,count_list)*model[word]
#         word_vec = model[word]#不加权重
        sentence.append(word_vec)
    #print(len(sentence))
    # 计算加权词向量
    for i in range(250):
        temp = 0
        for j in sentence:
            temp += j[i]
        temp = temp/len(sentence) 
        doc_vec.append(temp)
    return doc_vec

In [8]:
def lightGBM():
    print ("lightgbm")
    print (train_x.shape)
    print (train_y)
    train = lgb.Dataset(train_x, train_y, free_raw_data=False)
    eval = lgb.Dataset(valid_x, valid_y, free_raw_data=False)
    params = {
        'boosting_type': 'gbdt',
        'boosting': 'dart',
        'learning_rate': 0.15,
        'num_leaves': 100,
        'max_depth': 30,
        'min_data_in_leaf': 50,
        'feature_fraction': 0.1,
        'bagging_fraction': 0.5,
        'bagging_freq': 1,
        'lambda_l1': 0,
        'lambda_l2': 0,
        'min_split_gain': 0.5,
        'objective': 'binary',
        'metric': 'binary_logloss',
    }
    model = lgb.train(params,  # 参数字典
                      train,  # 训练集
                      num_boost_round=1000,  # 迭代次数
                      valid_sets=[eval, train],  # 验证集
                      early_stopping_rounds=50
                      )  # 早停系数

    y_pred = model.predict(test_arrays)
    #
    # Compute ROC curve and ROC area for each class
#     fpr, tpr, threshold = roc_curve(test_labels, y_pred)  ###计算真正率和假正率
#     roc_auc = auc(fpr, tpr)  ###计算auc的值

    for i in range(0, len(y_pred)):
        if y_pred[i] >= 0.5:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
        # y_pred = [list(x).index(max(x)) for x in y_pred]
    print(accuracy_score(test_labels, y_pred))
    print(classification_report(test_labels, y_pred))
    print(metrics.confusion_matrix(test_labels, y_pred))
    #FPR = FP/(FP+TN)
    cnf_matrix=metrics.confusion_matrix(test_labels, y_pred)
    FP=cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FP = FP.astype(float)
    print(FP)
    print(metrics.roc_auc_score(test_labels, y_pred))

In [9]:
from gensim.models import word2vec
import datetime
import gensim
print(datetime.datetime.now())
sentences = word2vec.Text8Corpus('CSIC2010_1/没把&符号消掉的/All.txt')
# model = word2vec.Word2Vec(sentences, min_count=1, size=300, window=7, workers=4,iter=100)
# model.save("model/mode300_epochs1000")
model = gensim.models.Word2Vec.load("model/mode300_epochs1000")

2020-06-12 18:02:20.083268


In [10]:
print(datetime.datetime.now())
len(model['http'])

2020-06-12 18:02:21.955681


d:\postgraduate\software\anaconda\envs\learn\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


300

In [11]:
from nltk.corpus import stopwords
from gensim.parsing import PorterStemmer
from collections import Counter
import string
import string
import nltk
anomaly_path = "CSIC2010_1/没把&符号消掉的/anomalousTrafficTestFeature.txt"
normal_path = "CSIC2010_1/没把&符号消掉的/NormalAll.txt"
countlist = get_countlist('CSIC2010_1/没把&符号消掉的/All.txt')

In [12]:
dj = {}
dn = {}
da = {}
# normal testing data
normal_file = open(normal_path, 'r',encoding='utf-8')
normal_lines = normal_file.readlines()
for text in normal_lines:
    text = text.split()
    for word in text:
        if word not in dn:            
            dn[word]=n_containing(word, countlist)
            dj=dn.copy()
# anomaly testing data
anomaly_file = open(anomaly_path, 'r',encoding='utf-8')
anomaly_lines = anomaly_file.readlines()
for text in anomaly_lines:
    text = text.split()
    for word in text:
        if word not in da:            
            da[word]=n_containing(word, countlist) 
dj.update(da)

In [13]:
len(dj)

54630

In [16]:
normal_test = []
anomaly_test = []
y_normal = []
y_anomaly = []
for text in normal_lines:
   # print(text.split())
    x_vec = getVector(text,model,countlist)
    y_normal.append(0)
    normal_test.append(x_vec)
print("normal_test=", len(normal_test))
for text in anomaly_lines:
    x_vec = getVector(text,model,countlist)
    y_anomaly.append(1)
    anomaly_test.append(x_vec)
print("anomaly_test=", len(anomaly_test))  

d:\postgraduate\software\anaconda\envs\learn\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


normal_test= 72000
anomaly_test= 25065


In [17]:
import numpy as np
x_all_test = np.append(normal_test,anomaly_test,axis=0)#normal_test，anomaly_test是二维的，axis=0表示沿行方向添加
y_all_test = np.append(y_normal,y_anomaly)

In [18]:
from sklearn.model_selection import train_test_split
train_arrays, test_arrays, train_labels, test_labels = train_test_split(x_all_test, y_all_test, test_size=0.1,
                                                                            random_state=20)
train_x, valid_x, train_y, valid_y = train_test_split(train_arrays, train_labels, test_size=0.1,
                                                          random_state=20)

In [19]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import time
start_time = time.time()  # 记录程序开始运行时间
lightGBM()
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))
print(datetime.datetime.now())

lightgbm
(78622, 250)
[1 1 1 ... 1 1 0]
[1]	training's binary_logloss: 0.471476	valid_0's binary_logloss: 0.468831


d:\postgraduate\software\anaconda\envs\learn\lib\site-packages\lightgbm\callback.py:188: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[2]	training's binary_logloss: 0.402165	valid_0's binary_logloss: 0.402745
[3]	training's binary_logloss: 0.349325	valid_0's binary_logloss: 0.351774
[4]	training's binary_logloss: 0.309213	valid_0's binary_logloss: 0.312722
[5]	training's binary_logloss: 0.275763	valid_0's binary_logloss: 0.280139
[6]	training's binary_logloss: 0.248649	valid_0's binary_logloss: 0.25362
[7]	training's binary_logloss: 0.226314	valid_0's binary_logloss: 0.23181
[8]	training's binary_logloss: 0.237221	valid_0's binary_logloss: 0.242185
[9]	training's binary_logloss: 0.216064	valid_0's binary_logloss: 0.221535
[10]	training's binary_logloss: 0.198624	valid_0's binary_logloss: 0.204673
[11]	training's binary_logloss: 0.183198	valid_0's binary_logloss: 0.189604
[12]	training's binary_logloss: 0.188661	valid_0's binary_logloss: 0.195109
[13]	training's binary_logloss: 0.174398	valid_0's binary_logloss: 0.181363
[14]	training's binary_logloss: 0.161433	valid_0's binary_logloss: 0.16872
[15]	training's binary_

[110]	training's binary_logloss: 0.0579071	valid_0's binary_logloss: 0.0734039
[111]	training's binary_logloss: 0.0564697	valid_0's binary_logloss: 0.0720445
[112]	training's binary_logloss: 0.0553141	valid_0's binary_logloss: 0.0710731
[113]	training's binary_logloss: 0.0540302	valid_0's binary_logloss: 0.0701327
[114]	training's binary_logloss: 0.0529852	valid_0's binary_logloss: 0.069386
[115]	training's binary_logloss: 0.0519551	valid_0's binary_logloss: 0.0686518
[116]	training's binary_logloss: 0.050949	valid_0's binary_logloss: 0.0676622
[117]	training's binary_logloss: 0.0513823	valid_0's binary_logloss: 0.0679998
[118]	training's binary_logloss: 0.0502552	valid_0's binary_logloss: 0.0671248
[119]	training's binary_logloss: 0.0506972	valid_0's binary_logloss: 0.067534
[120]	training's binary_logloss: 0.0512506	valid_0's binary_logloss: 0.0680464
[121]	training's binary_logloss: 0.0517503	valid_0's binary_logloss: 0.0684948
[122]	training's binary_logloss: 0.052323	valid_0's bin

[216]	training's binary_logloss: 0.0352001	valid_0's binary_logloss: 0.0543557
[217]	training's binary_logloss: 0.0346454	valid_0's binary_logloss: 0.0537452
[218]	training's binary_logloss: 0.0342492	valid_0's binary_logloss: 0.0533953
[219]	training's binary_logloss: 0.0344896	valid_0's binary_logloss: 0.0535802
[220]	training's binary_logloss: 0.0340184	valid_0's binary_logloss: 0.0531235
[221]	training's binary_logloss: 0.0342629	valid_0's binary_logloss: 0.0533296
[222]	training's binary_logloss: 0.0338292	valid_0's binary_logloss: 0.0530122
[223]	training's binary_logloss: 0.0333731	valid_0's binary_logloss: 0.0526073
[224]	training's binary_logloss: 0.0336359	valid_0's binary_logloss: 0.0528537
[225]	training's binary_logloss: 0.0339201	valid_0's binary_logloss: 0.0531095
[226]	training's binary_logloss: 0.0341578	valid_0's binary_logloss: 0.0533112
[227]	training's binary_logloss: 0.0343337	valid_0's binary_logloss: 0.0534763
[228]	training's binary_logloss: 0.0338985	valid_0's

...

[531]	training's binary_logloss: 0.0230057	valid_0's binary_logloss: 0.0409031
[532]	training's binary_logloss: 0.0229695	valid_0's binary_logloss: 0.0408099
[533]	training's binary_logloss: 0.0230215	valid_0's binary_logloss: 0.0408666
[534]	training's binary_logloss: 0.0229698	valid_0's binary_logloss: 0.0407498
[535]	training's binary_logloss: 0.0229124	valid_0's binary_logloss: 0.0407184
[536]	training's binary_logloss: 0.0229621	valid_0's binary_logloss: 0.040764
[537]	training's binary_logloss: 0.0229177	valid_0's binary_logloss: 0.0406469
[538]	training's binary_logloss: 0.0228739	valid_0's binary_logloss: 0.0405951
[539]	training's binary_logloss: 0.022924	valid_0's binary_logloss: 0.0406468
[540]	training's binary_logloss: 0.0229737	valid_0's binary_logloss: 0.040687
[541]	training's binary_logloss: 0.0230311	valid_0's binary_logloss: 0.0407426
[542]	training's binary_logloss: 0.0229925	valid_0's binary_logloss: 0.0407204
[543]	training's binary_logloss: 0.0230406	valid_0's bi

[638]	training's binary_logloss: 0.0229437	valid_0's binary_logloss: 0.0403282
[639]	training's binary_logloss: 0.0229905	valid_0's binary_logloss: 0.0403765
[640]	training's binary_logloss: 0.0229163	valid_0's binary_logloss: 0.0401762
[641]	training's binary_logloss: 0.0229628	valid_0's binary_logloss: 0.0402196
[642]	training's binary_logloss: 0.0230075	valid_0's binary_logloss: 0.0402624
[643]	training's binary_logloss: 0.0229524	valid_0's binary_logloss: 0.040177
[644]	training's binary_logloss: 0.0229956	valid_0's binary_logloss: 0.0402239
[645]	training's binary_logloss: 0.0229245	valid_0's binary_logloss: 0.040118
[646]	training's binary_logloss: 0.0228606	valid_0's binary_logloss: 0.0401113
[647]	training's binary_logloss: 0.0228438	valid_0's binary_logloss: 0.040052
[648]	training's binary_logloss: 0.0228125	valid_0's binary_logloss: 0.0400444
[649]	training's binary_logloss: 0.0228549	valid_0's binary_logloss: 0.0400832
[650]	training's binary_logloss: 0.0229005	valid_0's bi

[745]	training's binary_logloss: 0.0228575	valid_0's binary_logloss: 0.0403528
[746]	training's binary_logloss: 0.0228933	valid_0's binary_logloss: 0.040381
[747]	training's binary_logloss: 0.0228266	valid_0's binary_logloss: 0.0403903
[748]	training's binary_logloss: 0.0227811	valid_0's binary_logloss: 0.0403195
[749]	training's binary_logloss: 0.0228213	valid_0's binary_logloss: 0.0403571
[750]	training's binary_logloss: 0.0227937	valid_0's binary_logloss: 0.0404088
[751]	training's binary_logloss: 0.0227454	valid_0's binary_logloss: 0.0403896
[752]	training's binary_logloss: 0.0227867	valid_0's binary_logloss: 0.0404316
[753]	training's binary_logloss: 0.0227453	valid_0's binary_logloss: 0.0402567
[754]	training's binary_logloss: 0.0227802	valid_0's binary_logloss: 0.0402922
[755]	training's binary_logloss: 0.0227437	valid_0's binary_logloss: 0.0402855
[756]	training's binary_logloss: 0.0227797	valid_0's binary_logloss: 0.040324
[757]	training's binary_logloss: 0.0228194	valid_0's b

[849]	training's binary_logloss: 0.0225641	valid_0's binary_logloss: 0.0399708
[850]	training's binary_logloss: 0.0225148	valid_0's binary_logloss: 0.0398967
[851]	training's binary_logloss: 0.0225513	valid_0's binary_logloss: 0.0399346
[852]	training's binary_logloss: 0.0225109	valid_0's binary_logloss: 0.0399505
[853]	training's binary_logloss: 0.0224487	valid_0's binary_logloss: 0.0398898
[854]	training's binary_logloss: 0.0224807	valid_0's binary_logloss: 0.0399217
[855]	training's binary_logloss: 0.0224257	valid_0's binary_logloss: 0.0398461
[856]	training's binary_logloss: 0.0223906	valid_0's binary_logloss: 0.0397998
[857]	training's binary_logloss: 0.0224225	valid_0's binary_logloss: 0.0398301
[858]	training's binary_logloss: 0.0224595	valid_0's binary_logloss: 0.0398717
[859]	training's binary_logloss: 0.022495	valid_0's binary_logloss: 0.0399155
[860]	training's binary_logloss: 0.0224661	valid_0's binary_logloss: 0.0398175
[861]	training's binary_logloss: 0.0224314	valid_0's 

[953]	training's binary_logloss: 0.0223281	valid_0's binary_logloss: 0.0391677
[954]	training's binary_logloss: 0.0223545	valid_0's binary_logloss: 0.0391953
[955]	training's binary_logloss: 0.0223123	valid_0's binary_logloss: 0.0391477
[956]	training's binary_logloss: 0.0222513	valid_0's binary_logloss: 0.0392111
[957]	training's binary_logloss: 0.0222198	valid_0's binary_logloss: 0.03922
[958]	training's binary_logloss: 0.0222443	valid_0's binary_logloss: 0.0392383
[959]	training's binary_logloss: 0.0222332	valid_0's binary_logloss: 0.0392812
[960]	training's binary_logloss: 0.0221779	valid_0's binary_logloss: 0.0392188
[961]	training's binary_logloss: 0.0221513	valid_0's binary_logloss: 0.039183
[962]	training's binary_logloss: 0.0221333	valid_0's binary_logloss: 0.0391757
[963]	training's binary_logloss: 0.0220799	valid_0's binary_logloss: 0.0390661
[964]	training's binary_logloss: 0.0221068	valid_0's binary_logloss: 0.0390917
[965]	training's binary_logloss: 0.0220797	valid_0's bi

In [20]:
def catboost(train_arrays,train_labels,test_arrays,test_labels):
    import numpy as np
    from catboost import CatBoostClassifier, CatBoostRegressor, Pool
    model = CatBoostClassifier(iterations=200, depth=10, learning_rate=0.5, loss_function='Logloss', logging_level='Verbose',one_hot_max_size=100,l2_leaf_reg=5)
    model.fit(train_arrays, train_labels)
    y_pred = model.predict(test_arrays)
    preds_proba = model.predict_proba(test_arrays)
    print (preds_proba)
    for i in range(0, len(y_pred)):
        if y_pred[i] >= 0.5:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
        # y_pred = [list(x).index(max(x)) for x in y_pred]
    print(accuracy_score(test_labels, y_pred))
    print(classification_report(test_labels, y_pred))
    print(metrics.confusion_matrix(test_labels, y_pred))
    #FPR = FP/(FP+TN)
    cnf_matrix=metrics.confusion_matrix(test_labels, y_pred)
    FP=cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FP = FP.astype(float)
    print(FP)
    print(metrics.roc_auc_score(test_labels, y_pred))

In [21]:
import time
start_time = time.time()  # 记录程序开始运行时间
catboost(train_arrays,train_labels,test_arrays,test_labels)
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))
print(datetime.datetime.now())

0:	learn: 0.2687168	total: 1.68s	remaining: 5m 33s
1:	learn: 0.1964719	total: 3.28s	remaining: 5m 24s
2:	learn: 0.1712817	total: 4.88s	remaining: 5m 20s
3:	learn: 0.1515831	total: 6.43s	remaining: 5m 15s
4:	learn: 0.1392760	total: 8s	remaining: 5m 12s
5:	learn: 0.1310535	total: 9.64s	remaining: 5m 11s
6:	learn: 0.1189706	total: 11.3s	remaining: 5m 10s
7:	learn: 0.1127736	total: 13s	remaining: 5m 11s
8:	learn: 0.1070454	total: 14.6s	remaining: 5m 10s
9:	learn: 0.0995577	total: 16.2s	remaining: 5m 8s
10:	learn: 0.0954991	total: 17.9s	remaining: 5m 7s
11:	learn: 0.0908969	total: 19.5s	remaining: 5m 5s
12:	learn: 0.0878679	total: 21s	remaining: 5m 2s
13:	learn: 0.0845151	total: 22.6s	remaining: 4m 59s
14:	learn: 0.0808722	total: 24.2s	remaining: 4m 57s
15:	learn: 0.0787989	total: 25.7s	remaining: 4m 55s
16:	learn: 0.0752299	total: 27.3s	remaining: 4m 53s
17:	learn: 0.0725311	total: 28.8s	remaining: 4m 51s
18:	learn: 0.0705162	total: 30.3s	remaining: 4m 49s
19:	learn: 0.0681675	total: 31.9s

156:	learn: 0.0178190	total: 4m 3s	remaining: 1m 6s
157:	learn: 0.0177942	total: 4m 5s	remaining: 1m 5s
158:	learn: 0.0177686	total: 4m 6s	remaining: 1m 3s
159:	learn: 0.0177332	total: 4m 8s	remaining: 1m 2s
160:	learn: 0.0177059	total: 4m 9s	remaining: 1m
161:	learn: 0.0176757	total: 4m 11s	remaining: 58.9s
162:	learn: 0.0176369	total: 4m 12s	remaining: 57.3s
163:	learn: 0.0176221	total: 4m 14s	remaining: 55.8s
164:	learn: 0.0176046	total: 4m 15s	remaining: 54.2s
165:	learn: 0.0175697	total: 4m 17s	remaining: 52.7s
166:	learn: 0.0175486	total: 4m 18s	remaining: 51.1s
167:	learn: 0.0175323	total: 4m 20s	remaining: 49.6s
168:	learn: 0.0175043	total: 4m 21s	remaining: 48s
169:	learn: 0.0174934	total: 4m 23s	remaining: 46.5s
170:	learn: 0.0174679	total: 4m 25s	remaining: 44.9s
171:	learn: 0.0174317	total: 4m 26s	remaining: 43.4s
172:	learn: 0.0174284	total: 4m 28s	remaining: 41.8s
173:	learn: 0.0174074	total: 4m 29s	remaining: 40.3s
174:	learn: 0.0174053	total: 4m 31s	remaining: 38.8s
175